In [ ]:
!pip install transformers[torch] datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.3 MB/s eta 0:00:00




*  Dataset
*  Tokenizer
*  Model

In [ ]:
!git clone https://github.com/tatsu-lab/stanford_alpaca

Cloning into 'stanford_alpaca'...
remote: Enumerating objects: 129, done.
remote: Total 129 (delta 0), reused 0 (delta 0), pack-reused 129
Receiving objects: 100% (129/129), 9.15 MiB | 8.55 MiB/s, done.
Resolving deltas: 100% (62/62), done.


In [ ]:
!ls stanford_alpaca/

alpaca_data.json	 LICENSE	   train.py
assets			 model_card.md	   utils.py
configs			 prompt.txt	   WEIGHT_DIFF_LICENSE
DATA_LICENSE		 README.md	   weight_diff.py
datasheet.md		 requirements.txt
generate_instruction.py  seed_tasks.jsonl


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling
from rich import print

In [ ]:
model_name = "distilgpt2"
#model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
from datasets import load_dataset
alpaca_dataset = load_dataset("json",data_files="stanford_alpaca/alpaca_data.json")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
shuffled_alpaca_dataset = alpaca_dataset.shuffle(seed=42)

In [ ]:
prompt_template_with_input = "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n \
### Instruction:\n \
{instruction}\n \
### Input:\n \
{input}\n \
### Response:\n \
{response}"

prompt_template_without_input = "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n \
### Instruction:\n \
{instruction}\n \
### Response:\n \
{response}"

def add_content(example):
  if example["input"]:
    prompt = prompt_template_with_input.format(instruction=example["instruction"],input=example["input"],response=example["output"]) + tokenizer.eos_token
  else:
    prompt = prompt_template_without_input.format(instruction=example["instruction"],response=example["output"]) + tokenizer.eos_token
  return {"prompt" : prompt}

In [ ]:
alpaca_train_prompt = shuffled_alpaca_dataset.map(add_content)

Map:   0%|          | 0/52002 [00:00<?, ? examples/s]

In [ ]:
print(alpaca_train_prompt["train"][10]["prompt"])

Below is an instruction that describes a task, paired with an input that provides further context. Write a response
that appropriately completes the request.
 ### Instruction:
 Arrange the following countries in ascending order of GDP per capita.
 ### Input:
 China, US, India, Japan
 ### Response:
 India, China, Japan, US<|endoftext|>

In [ ]:
def tokenizer_function(example):
  outputs = tokenizer(
        example["prompt"],
        truncation=True,
        padding=True,
        return_tensors="pt"
    ).to("cuda")

  return outputs

In [ ]:
train_tokenized_data = alpaca_train_prompt.map(tokenizer_function, batched=True)
train_tokenized_data = train_tokenized_data.remove_columns(alpaca_train_prompt["train"].column_names)

Map:   0%|          | 0/52002 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer,mlm=False)

In [ ]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    output_dir="ft-ds",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    #evaluation_strategy="steps",
    #eval_steps=1_0,
    logging_steps=1_00,
    gradient_accumulation_steps=8,
    num_train_epochs=10,
    weight_decay=0.1,
    warmup_steps=1_000,
    lr_scheduler_type="cosine",
    learning_rate=5e-4,
    save_steps=1_00,
    fp16=True
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=train_tokenized_data["train"].select(range(0,20000)),
    # eval_dataset=train_tokenized_data["test"].select(range(0,100)),
)


In [148]:
trainer.train()

Step,Training Loss
100,0.670100
200,0.679800
300,0.700600
400,0.690200
500,0.724000
600,0.772900
700,0.728400
800,0.770300
900,0.823600
1000,0.789000


TrainOutput(global_step=3120, training_loss=0.6090948895002023, metrics={'train_runtime': 2857.2796, 'train_samples_per_second': 69.997, 'train_steps_per_second': 1.092, 'total_flos': 4.022572021501133e+16, 'train_loss': 0.6090948895002023, 'epoch': 9.98})

In [149]:
from transformers import pipeline
from rich import print
import warnings
warnings.filterwarnings("ignore")

In [150]:
pipe = pipeline(
    "text-generation", model=model, tokenizer=tokenizer, device="cuda"
)

In [151]:
prompt_template_with_input_no_response = "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n \
### Instruction:\n \
{instruction}\n \
### Input:\n \
{input}\n \
### Response:\n"

prompt_template_without_input_no_repsonse = "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n \
### Instruction:\n \
{instruction}\n \
### Response:\n \
"

In [152]:
def generate_prompt(example):
  if example["input"]:
    prompt = prompt_template_with_input_no_response.format(instruction=example["instruction"],input=example["input"])
  else:
    prompt = prompt_template_without_input_no_repsonse.format(instruction=example["instruction"])
  return prompt

In [163]:
index = 22000
input = generate_prompt(shuffled_alpaca_dataset["train"][index])
print(input)
print(add_content(shuffled_alpaca_dataset["train"][index])["prompt"])

Below is an instruction that describes a task, paired with an input that provides further context. Write a response
that appropriately completes the request.
 ### Instruction:
 Classify the following paragraph as formal or informal.
 ### Input:
 Hey there! I was just wondering if you were available to meet up for lunch tomorrow?
 ### Response:

Below is an instruction that describes a task, paired with an input that provides further context. Write a response
that appropriately completes the request.
 ### Instruction:
 Classify the following paragraph as formal or informal.
 ### Input:
 Hey there! I was just wondering if you were available to meet up for lunch tomorrow?
 ### Response:
 Informal<|endoftext|>

In [164]:
print(pipe(input, eos_token_id=tokenizer.eos_token_id,pad_token_id=tokenizer.eos_token_id,max_new_tokens=200, do_sample=True,temperature=0.5,top_p=0.9)[0]["generated_text"].split(input)[1])

Informal. This is informal. It is typically used for informal conversations, while informal is more informal. For 
informal conversations, the host may use a laptop or other means of communication to listen and to express their 
opinion. It is typically done conversatively and courteous even when there is an interrogative question. It is 
commonly used in informal settings, such as in academic classes or formal meetings. It is also common in the 
context of informal language and often places, such as in a conference room. This could be classified as informal, 
informal, or informal. It is also very informal in academic or formal communication. It's a good way to meet the 
needs of informal people. It's possible for informal and informal conversations to be conversational and not just 
polite. For instance, if you're looking for a friendly and friendly gesture, then you should look for some informal
conversation that you can use to express your opinions. It may also be used to listen to other people's opinions, 
to

In [ ]:
non_finetuned_pipe = pipeline(
    "text-generation", "gpt2", device="cuda"
)

In [ ]:
print("Non fine-tuned Prediction: %s"%non_finetuned_pipe(input,eos_token_id=tokenizer.eos_token_id,max_new_tokens=2, do_sample=True,temperature=0.5)[0]["generated_text"].split(input)[1])
print("Label: %s"%label)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Non fine-tuned Prediction:  I've

Label: Postitive

In [158]:
model.save_pretrained("output_folder")